In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam

In [2]:
sentence = "I just want you for my own More than you could ever know Make my wish come true All I want for Christmas is you"
sequence_length = 10 # 하나의 긴 문장을 한번에 예측하는 것이 아니라, 일정한 길이를 가진 짧은 문장을 계속 예측하는 문제로 변경
char_set = list(set(sentence))
char_dic = {c:i for i, c in enumerate(char_set)}
x_data = []
y_data = []
for i in range(len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1:i + 1 + sequence_length]
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])
x_one_hot = np.array([np.eye(len(char_set))[x] for x in x_data])
X = torch.Tensor(x_one_hot)
y = torch.LongTensor(y_data)

In [3]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, layers):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        
        return x

In [4]:
model = Model(len(char_set), len(char_set), 2)

In [5]:
with torch.no_grad():
    preds = model(X)
    predicted_str = sentence[0]
    results = preds.argmax(dim=2)
    for i, result in enumerate(results):
        if i == 0:
            predicted_str += ''.join(char_set[i] for i in result)
        else:
            predicted_str += char_set[result[-1]]
    
    print(predicted_str)

IuueeeuuelluueeeueleueueeuluueeCennlCueeeueeeuuuCeCCCuluuuuuuCCfueeleeeCeeuenueueCnuueuelluuefuueeeleelleeeeeee


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

In [8]:
for i in range(1000):
    optimizer.zero_grad()
    preds = model(X)
    loss = criterion(preds.view(-1, len(char_set)), y.view(-1))
    loss.backward()
    optimizer.step()

In [9]:
with torch.no_grad():
    preds = model(X)
    predicted_str = sentence[0]
    results = preds.argmax(dim=2)
    for i, result in enumerate(results):
        if i == 0:
            predicted_str += ''.join(char_set[i] for i in result)
        else:
            predicted_str += char_set[result[-1]]
    
    print(predicted_str)

I wust want you for my own More than you could ever know Make my wish come true All I want for Christmas is you
